# Neural Networks

신경망은 torch.nn 패키지를 사용하여 생성할 수 있습니다. 지금까지 autograd를 살펴봤는데요. nn은 모델을 정의하고 미분하는데 autograd를 사용합니다. nn.Module은 계층(layer)와 output을 반환하는 foraward(input) 메서드를 포함하고 있습니다.

### Conv-net
간단한 Feed-forward network입니다. 입력을 받아 여러계층에 전달한 후 ouput을 제공합니다.
학습의 과정은 다음과 같습니다.
- 학습 가능한 parameter를 갖는 신경망을 정의합니다.
- 데이터셋(dataset) 입력을 반복합니다.
- 입력을 신경망에서 처리합니다.
- 손실을 계산합니다.
- Gradient를 신경망의 배개변수들에 역으로 전파합니다.
- 신경망의 가중치를 갱신합니다 : Weight* = Weight - (learnign rate x gradient)

### torch.nn
torch.nn은 미니배치만 지원합니다. 즉 torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니매치만을 입력으로 받습니다. 예를들어 nnConv2D는 nsamples x nChannels x Height x Width 의 4차원 Tensor를 입력으로 합니다.

만약 하나의 샘플만 있다면, input.unsqueeze(0)을 사용해서 가짜 차원을 추가해 봅시다.
- torch.Tensor : backward() 같은 autograd 연산을 지원하는 다차원 배열입니다. tensor에 대한 gradient를 가지게 됩니다.
- nn.Module : 신경망 모듈로 매개변수를 Encapsulation 하는 간단한 방법으로, GPU로 이동, exporting, loading 등의 작업을 위한 도움을 줍니다.
- nn.Parameter : Tensor 의 한 종류로 Module에 속성으로 할당될 떄 자동으로 매개변수로 등록됩니다.
- autograd.Function : autograd 연산의 전방향과 역방향 정의를 구현합니다. 모든 tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor 를 생성하고 History를 encoding 하는 함수들과 연결되어 있습니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Net(nn.Module):
    def __init__(self):
        # super 은 일반적으로 부모 클래스의 init함수를 쓰고 싶을때다.
        # __init__은 클래스 인스턴스 생성과 초기값 입력을 한번에 처리가 가능하지!
        super(Net, self).__init__()
        # 하나의 input image channel, 6개 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # affine operation : y = Wx * b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        #max pooling (2,2) window에 해주자
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        # batch dimension을 빼고 모든 dimension을 호출한다.
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [6]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward 함수만 정의하고 나면, 변화도를 계산하는 backward 함수는 autograd 를 사용하여 자동으로 정의됩니다. forward 함수에서는 어떠한 Tensor 연산을 사용해도 괜찮습니다.

In [9]:
params = list(net.parameters())
print(len(parmas), params[0].size())

10 torch.Size([6, 1, 5, 5])


In [13]:
test_input = torch.randn(1,1,32,32)
print(test_input)
out = net(test_input)
print("")
print(out)

tensor([[[[ 1.1776,  0.7962, -1.3007,  ..., -0.1552,  1.1180,  0.4885],
          [ 1.6221,  0.0040, -0.9108,  ..., -0.3684, -0.8328,  0.0202],
          [-0.7043, -2.1098,  1.4173,  ...,  0.9286, -1.1388, -1.3099],
          ...,
          [ 0.4142, -0.4618, -0.2915,  ...,  0.0038,  0.1195,  2.1477],
          [ 0.2168,  0.1630, -1.0339,  ...,  1.4760,  1.4963, -0.6476],
          [-1.2849, -0.5145,  1.5898,  ..., -0.2405,  0.0347, -1.0495]]]])

tensor([[-0.0419,  0.0123,  0.0572, -0.0887, -0.1177,  0.0516, -0.1555,  0.0903,
          0.0227,  0.0952]], grad_fn=<ThAddmmBackward>)


In [11]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [12]:
out

tensor([[-0.0517,  0.0044,  0.0425, -0.0849, -0.1377,  0.0557, -0.1517,  0.0973,
          0.0070,  0.0709]], grad_fn=<ThAddmmBackward>)

### Loss Function

손실합수는 (output, target)을 한 쌍으로 입력받아, output과 target의 distance를 계산합니다.
- nn.MSEloss : mean-squared error

In [16]:
ouput = net(test_input)
# dummy target을 만듭니다.
target = torch.arange(1,11)
# output과 같은 shape을 하나 만들어둡니다.
target = target.view(1,-1)
# LongTensor 에서 FloatTensor로 바꾸어줍니다.
target = target.type(torch.FloatTensor)
criterion = nn.MSELoss()

loss = criterion(ouput, target)
print(loss)

tensor(38.4450, grad_fn=<MseLossBackward>)


이제 ``.grad_fn`` 속성을 사용하여 ``loss``를 역방향으로 따라가 봅시다.

```
    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss
```

따라서 ``loss.backward()``를 실행할 때, 전체 그래프는 손실(loss)에 대해 미분되며, 그래프 내의 ``requires_grad= True``인 모든 Tensor는 Gradient가 누적된 ``.grad`` Tensor를 갖게 됩니다.:


In [18]:
print("MSELoss:", loss.grad_fn)
print("linear:", loss.grad_fn.next_functions[0][0])
print("ReLU:", loss.grad_fn.next_functions[0][0].next_functions[0][0])

MSELoss: <MseLossBackward object at 0x1175238d0>
linear: <ThAddmmBackward object at 0x117523898>
ReLU: <ExpandBackward object at 0x1175238d0>


### 역전파 (BackProp)

오차(error)를 역전파하기 위해 할 일은 loss.backword()가 전부입니다. 기존의 변화도를 지우는 작업이 필요합니다. 이제 loss.backward()를 호출하여, 역전파 전후의 bias gradient를 살펴봅시다.

In [19]:
net.zero_grad()
print("before backward: ",net.conv1.bias.grad)
loss.backward()
print('after backward: ', net.conv1.bias.grad)

before backward:  tensor([0., 0., 0., 0., 0., 0.])
after backward:  tensor([-0.0108, -0.1272, -0.0131, -0.0437,  0.1452, -0.0726])


### Weight 갱신

```
    weight* = wegiht - (learning rate * gradient)
```


In [24]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [25]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr= 0.01)
optimizer.zero_grad()
output = net(test_input)
loss = criterion(output, target)
loss.backward()
optimizer.step()